<a href="https://colab.research.google.com/github/Shama09/files/blob/master/ML2_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

In [ ]:
import lightgbm as lgbm

In [ ]:
with open('sample_data/additional_features.json') as f:
    ad_data = json.load(f)

In [ ]:
categories = []
df_index = []

for i in range(len(ad_data)):
  categories.append(ad_data[i]['categories'])
  df_index.append(ad_data[i]['index'])

In [ ]:
df_cat = pd.DataFrame(columns=['categories'])
for i in range(df.shape[0]):
  df_cat.at[i, 'categories'] = []

In [ ]:
for i in range(len(df_index)):
  df_cat.at[df_index[i], 'categories'] = categories[i]

In [ ]:
df_cat.to_csv('cat.csv', index=False)

In [ ]:
#df = pd.read_json('time_series_train.json', orient='columns')

# Data to csv

In [ ]:
#a, b = df.shape
#data = np.empty((a, b))
#data[:] = np.nan

for i in range(a):
    for j in range(b):
        if (df.iloc[i][j] == None):
            break
        data[i, df.iloc[i][j][0]] = df.iloc[i][j][1]

In [ ]:
#pd.DataFrame(data).to_csv('dataml2hw1.csv',  header=None, index=None)

# Analysis

In [ ]:
df = pd.read_csv('dataml2hw1.csv', header=None)

In [ ]:
df.shape[0] - df.iloc[:, 90:].isna().any(axis=1).sum()

25439

In [ ]:
train = df[~df.iloc[:, 90:].isna().any(axis=1)].iloc[:, 90:].values

In [ ]:
X_train, X_test = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
n = 9
x = X_train[:, :n]
y = X_train[:, n]
for i in range(1, X_train.shape[1] - n):
    x = np.concatenate((x, X_train[:, i : n+i]), axis=0)
    y = np.concatenate((y, X_train[:, i + n]))
    print(x.shape)
    print(y.shape)

In [ ]:
x.shape

(20351, 9)

In [ ]:
model = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=5, random_state=0)

In [ ]:
model.fit(x[10000:15000], y[10000:15000])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [76]:
for depth in np.array([2, 5, 10, 20]):
  for n in np.array([5, 10, 50]):
    print(depth)
    print(n)
    model = GradientBoostingClassifier(n_estimators=n, learning_rate=1.0, max_depth=depth, random_state=0)
    model.fit(x[:20000], y[:20000])
    print(model.score(x[:20000], y[:20000]))
    print(mean_absolute_error(model.predict(x[:20000]), y[:20000]))
    print(model.score(x[20000:], y[20000:]))
    print(mean_absolute_error(model.predict(x[20000:]), y[20000:]))

2
5


KeyboardInterrupt: ignored

In [ ]:
model.score(x, y)

0.14957495946145152

In [ ]:
mean_absolute_error(model.predict(x), y)

287.5325045452312

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df.iloc[:, -n-1:])
test = imputer.transform(df.iloc[:, -n-1:])

In [ ]:
test[:, :-1].shape

(29860, 9)

In [ ]:
model.score(test[:, :-1], test[:, -1])

0.02886805090421969

In [ ]:
mean_absolute_error(model.predict(test[:, :-1]), test[:, -1])

300.45174146014733

In [ ]:
df_cat['categories'][0][1]

'EA'

In [ ]:
one = OneHotEncoder()
one.fit(df_cat)

TypeError: ignored

In [ ]:
df_cat['categories']

0                                [actions, EA, indie-game]
1                                                 [casual]
2                            [actions, casual, indie-game]
3                    [adventures, casual, indie-game, rpg]
4                                                       []
                               ...                        
29855                                                   []
29856            [actions, adventures, casual, indie-game]
29857    [actions, adventures, casual, indie-game, stra...
29858                                         [strategies]
29859                                                   []
Name: categories, Length: 29860, dtype: object

In [ ]:
pred1 = model.predict(test)

In [ ]:
pred2 = model.predict(np.concatenate((test[:, 1:], pred1.reshape(-1, 1)), axis=1))

In [ ]:
pred3 = model.predict(np.concatenate((test[:, 2:], pred1.reshape(-1, 1), pred2.reshape(-1, 1)), axis=1))

# To file

In [ ]:
Id = []
for i in range(df.shape[0]):
    Id.append(str(i)+str('_')+str(0))
    Id.append(str(i)+str('_')+str(1))
    Id.append(str(i)+str('_')+str(2))

In [ ]:
index = pd.DataFrame(Id, columns=['Id'])

In [ ]:
pred = np.concatenate((pred1, pred2, pred3), axis=0)

In [ ]:
index['Predicted'] = pred

In [ ]:
index

,Id,Predicted
0,0_0,4.0
1,0_1,28.0
2,0_2,16.0
3,1_0,28.0
4,1_1,9.0
...,...,...
89575,29858_1,5.0
89576,29858_2,38.0
89577,29859_0,5.0
89578,29859_1,53.0


In [ ]:
index.to_csv('pred.csv', index=False)